In [1]:
import numpy as np

Define base field and matrix spaces:

In [2]:
K.<w> = CyclotomicField(3)

M3= MatrixSpace(K, 3,3)
M9= MatrixSpace(K, 9,9)
V9= MatrixSpace(K, 1,9)

w**3 == 1

True

Define Weyl Heisenberg basis in $d=3$:

In [3]:
# Define matrices S (also denoted by X) and W (also denoted by Z)


S = M3([[0,0,1],[1,0,0], [0,1,0]])
W = M3([[w**3,0,0],[0, w,0],[0,0, w**2]])
Id3 = M3(identity_matrix(3))
Id9 = M9(identity_matrix(9))
Sinv = S**2
Winv = W**2

# Define basis as list and a correpsonding dictionary

Basis = [Id3, S, W, S**2, W**2, S*W, (S**2)*(W**2), (S**2)*W, S*(W**2)]

Basis_dic = {0 : "Id3", 1 : "S", 2 : "W", 3 : "S^2", 4 : "W^2", 5 : "SW", 6 : "S^2 W^2", 7 : "S^2 W", 8 : "S W^2" }

Basis_dic[8]

'S W^2'

Define the qutrit basis vectors as a function that takes as input an integer $i$ and outputs $| i \rangle$ as a numpy array:

In [4]:
def unitV(i):
    vec = np.zeros(3)
    for l in range(0,3):
        if l == i:
            vec[l] = 1
    return vec

# Test 

vector(unitV(0))

(1.0, 0.0, 0.0)

Define the Fourier in dimension 3 and the CNOT gate:

In [5]:
# Fourier gate

F3 = M3([[1,1,1], [1,w,w**2], [1,w**2, w**4]])


# CNOT gate

L = []
for i in range(0,3):
    L.append(M3(vector(unitV(i)).tensor_product(vector(unitV(i)))).tensor_product(matrix(S**i)))

CNOT = matrix(sum(L))


Define a function that takes as input two subalgebras represented as lists of elements and checks if these subalgebras are quasi-orthogonal. In this case the function outputs true, otherwise it outputs false.

In [6]:
def is_quasi_orth(A,B):
    for V in A:
            for W in B:
                K = V*W
                if (K.trace())/(K.nrows()) != ((V.trace())*(W.trace()))/((V.nrows())*(W.nrows())):
                    return false
    return true


Now define a function that checks if a subalgebra is delocalised by taking as input a subalgebra $A$ and a list of local subalgebras and checks if $A$ is quasi-orhtogonal to each local subalgebra. The function outputs true, if this is the case and false otherwise.

In [7]:
def is_delocalised(A,List):
    for B in List:
        if is_quasi_orth(A,B) != true:
            return false
    return true


Define a function that computes the reshuffle of a matrix:

In [8]:
def matrix_reshuffle(M):
    matrix_reshaped = np.array(M).reshape((3,3,3,3))
    return matrix(np.array([[[[matrix_reshaped[l,j,k,i] for i in range(0,3)] for j in range(0,3)] for k in range(0,3)] for l in range(0,3)]).reshape((9,9)))


Define a function that checks if a matrix is dual unitary:

In [9]:
def is_dual_unitary(M):
    return matrix_reshuffle(M).is_unitary()

Define a function that computes the partial transpose of a matrix:

In [10]:
def matrix_partial_transpose(M):
    matrix_reshaped = np.array(M).reshape((3,3,3,3))
    return matrix(np.array([[[[matrix_reshaped[k,j,i,l] for i in range(0,3)] for j in range(0,3)] for k in range(0,3)] for l in range(0,3)]).reshape((9,9)))


Define a function that checks if a matrix is $\Gamma$-dual unitary:

In [11]:
def is_gamma_dual_unitary(M):
    return matrix_partial_transpose(M).is_unitary()

Define function thats checks 2-unitarity by checking if matrix itself, dual and gamma dual matrices are unitary:

In [12]:
def is_2unitary(M):
    return (M.is_unitary() and is_dual_unitary(M) and is_gamma_dual_unitary(M))

II. Matrices

In this section the relevant unitaries will be defined, their orders will be computed and it will be confirmed that they are indeed 2-unitary.

$U_{OLS}$ is the 2-unitary permutation matrix that is obtained from an OLS in normal form:

In [13]:
np.kron(unitV(0), unitV(0))

array([1., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
def Bell_basis(d):
    K.<w> = CyclotomicField(d)
    V = MatrixSpace(K, 1, d**2)
    L = []
    for i in range(0,d):
            L.append(vector(np.kron(unitV(i),unitV(i))))
    return V(sum(L))

Bell_basis(3)


[1 0 0 0 1 0 0 0 1]

In [15]:
U_OLS = M9(np.array([np.kron(unitV(0), unitV(0)), np.kron(unitV(1), unitV(1)),np.kron(unitV(2), unitV(2)), np.kron(unitV(2), unitV(1)),np.kron(unitV(0), unitV(2)),np.kron(unitV(1), unitV(0)), np.kron(unitV(1), unitV(2)),np.kron(unitV(2), unitV(0)), np.kron(unitV(0), unitV(1))]))

In [16]:
# check 2-unitarity

is_2unitary(U_OLS)

True

$U_{\Lambda}$ is the 2-unitary Hadamard matrix obtained from the doubly perfect sequence $\Lambda (a,b) = \omega_3^{a^2 + b^2}$:

In [17]:
U_Lambda = M9([[-2/3*w - 1/3    ,        0       ,     0      ,      0 , 1/3*w + 2/3    ,        0      ,      0,0 , 1/3*w + 2/3],
[           0 , 1/3*w - 1/3      ,      0      ,      0       ,     0 ,-2/3*w - 1/3, -2/3*w - 1/3   ,         0       ,     0],
[           0     ,       0 , 1/3*w - 1/3 ,-2/3*w - 1/3     ,       0     ,       0   ,         0 ,-2/3*w - 1/3      ,      0],
[           0      ,      0 ,-2/3*w - 1/3  ,1/3*w - 1/3      ,      0      ,      0    ,        0 ,-2/3*w - 1/3       ,     0],
[ 1/3*w + 2/3     ,       0      ,      0    ,        0 ,-2/3*w - 1/3     ,       0       ,     0    ,        0  ,1/3*w + 2/3],
[           0 ,-2/3*w - 1/3      ,      0    ,        0 ,           0  ,1/3*w - 1/3 ,-2/3*w - 1/3   ,         0 ,           0],
[           0 ,-2/3*w - 1/3       ,     0     ,       0  ,          0 ,-2/3*w - 1/3  ,1/3*w - 1/3    ,        0  ,          0],
[           0   ,         0 ,-2/3*w - 1/3 ,-2/3*w - 1/3       ,     0    ,        0    ,        0  ,1/3*w - 1/3    ,        0],
[ 1/3*w + 2/3    ,        0      ,      0  ,          0  ,1/3*w + 2/3    ,        0    ,        0  ,          0 ,-2/3*w - 1/3]])

In [18]:
# check 2-unitarity

is_2unitary(U_Lambda)

True

In [19]:
# order 3

U_Lambda**3

[1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]

$U_{CHM}$ is the 2-unitary Hadamard matrix taken from https://arxiv.org/pdf/1506.08857:

In [20]:
#U_CHM = (1/3)*M9([[1,1,1,1,w**2,w,1,w,w**2],[1,1,1,w,1,w**2,w**2,1,w],[1,1,1,w**2,w,1,w,w**2,1],[1,w,w**2,1,1,1,1,w**2,w],
        #[w,w**2,1,w**2,w**2,w**2,1,w**2,w], [w**2,1,w,w,w,w,1,w**2,w], [1,w**2,w,1,w,w**2,1,1,1], 
        #[w**2,w,1,1,w,w**2,w,w,w],[w,1,w**2,1,w,w**2,w**2,w**2,w**2]])
        
U_CHM = (1/3)*M9([[1,1,1,1,w,w**2,1,w**2,w],[1,1,1,w**2,1,w,w,1,w**2],[1,1,1,w,w**2,1,w**2,w,1],
                  [1,w,w**2,1,w**2,w,1,1,1],
        [w,w**2,1,1,w**2,w,w**2,w**2,w**2], [w**2,1,w,1,w**2,w,w,w,w], [1,w**2,w,1,1,1,1,w,w**2], 
        [w**2,w,1,w,w,w,1,w,w**2],[w,1,w**2,w**2,w**2,w**2,1,w,w**2]])


In [21]:
# test 2-unitarity

is_2unitary(U_CHM)

#same quasi-orthogonal subalgebras as Goyeneches Matrix (C_2,0 and C_1,0)

True

In [22]:
# order 4

U_CHM**4

[1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 1 0 0 0 0]

$U_{\Lambda_{CHM}}$ is the Hadamard matrix obtained from the doubly perfect sequence $\Lambda (a_1,a_2,b_1, b_2) = \omega^{a_1 a_2} \omega^{(a_1 - b_1)^2 + (a_2 - b_2)^2} \omega^{-b_1 b2} $

In [23]:
H = [[[[(w**(a1*a2))*(w**((a1-b1)**2 + (a2-b2)**2))*(w**(-b1*b2)) for a1 in range(0,3)] for a2 in range(0,3)] for b1 in range(0,3)] for b2 in range(0,3)]


U_LambdaCHM = (1/3)*matrix(np.array(H).reshape((9,9)))
           
# C1,2 and C2,2
        

In [24]:
# check 2-unitarity

is_2unitary(U_LambdaCHM)

True

In [25]:
# order 6

U_LambdaCHM**6


[1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1]

In [26]:
# traces

U_OLS.trace()

1

In [27]:
U_Lambda.trace()

-3

In [28]:
U_LambdaCHM.trace()

3

In [29]:
U_CHM.trace()

1

III. Quantum Orthogonal Latin Squares

In this section the unitaries from II. will be used to construct quantum orthogonal Latin squares.



Define a function that checks if a unitary defines a QOLS:

In [29]:
def is_QOLS(U):
        List = []
        for i in range(0,9):
            for j in range(0,9):
                List.append(U*M9(np.outer(np.kron(unitV(i),unitV(j)),np.kron(unitV(i),unitV(j))))*U.conjugate_transpose())
        return matrix(sum(List)) == Id9

# test

is_QOLS(U_OLS)


True

In [30]:
is_QOLS(U_Lambda)

True

In [31]:
is_QOLS(U_LambdaCHM)

True

In [32]:
is_QOLS(U_CHM)

True

Define a function that computes a quantum orthogonal Latin square by taking as input a matrix $U$ and outputting a list of vectors that form a QLS:

In [33]:
 def QOLS(M):
    output = []
    for i in range(0,3):
        for j in range(0,3):
            output.append((M*V9(vector(np.kron(unitV(i),unitV(j)))).transpose()).transpose())
    return output

# test

QOLS(U_OLS)

[[1 0 0 0 0 0 0 0 0],
 [0 0 0 0 0 0 0 0 1],
 [0 0 0 0 1 0 0 0 0],
 [0 0 0 0 0 1 0 0 0],
 [0 1 0 0 0 0 0 0 0],
 [0 0 0 0 0 0 1 0 0],
 [0 0 0 0 0 0 0 1 0],
 [0 0 0 1 0 0 0 0 0],
 [0 0 1 0 0 0 0 0 0]]

IV. Quasi-orthogonal Subalgebras

In this section the local subalgebras will be defined and the matrices from II. will be used to construct delocalised subalgebras.

Define left and right subalgebras as lists:

In [39]:
L = [ M9(Basis[i].tensor_product(Id3)) for i in range(0,9) ]
R = [ M9(Id3.tensor_product(Basis[i])) for i in range(0,9) ]

# combine them in a list

LocalSubal = [L,R]

Define the left and right subalgebras, $\mathcal{L}$ adn $\mathcal{R}$ respectively, conjugated by a unitary $U$ as functions that take as input a 2-unitary $U$ and output the algebras $\mathcal{A}_L := U \mathcal{L} U^\dagger$ and $\mathcal{A}_R := U \mathcal{R} U^\dagger$ as lists.

In [40]:
def AL(U):
    return [M9(U*L[i]*U.conjugate_transpose()) for i in range(0,9)]

def AR(U):
    return [M9(U*R[i]*U.conjugate_transpose()) for i in range(0,9)]

# define function that merges both subalgebras in a list

def rotated_subal(U):
    return [AL(U),AR(U)]



In [41]:
# Test if all subalgebras are delocalised

for A in rotated_subal(U_OLS):
        print(is_delocalised(A, LocalSubal))
  

True
True


In [42]:
for A in rotated_subal(U_Lambda):
        print(is_delocalised(A, LocalSubal))

True
True


In [43]:
for A in rotated_subal(U_CHM):
        print(is_delocalised(A, LocalSubal))

True
True


In [44]:
for A in rotated_subal(U_LambdaCHM):
        print(is_delocalised(A, LocalSubal))

True
True


VI. Explicit form of $\mathcal{A}_L := U \mathcal{L} U^\dagger$ and $\mathcal{A}_R := U \mathcal{R} U^\dagger$

In the following code will be provided that can be used to calculate the explicit form of the delocalises subalgebras in terms of the tensor product of the operators $S$ and $Z$:

In [50]:
def AR_dic(U):
    output_list = []
    for M in AR(U):
        for j in range(0,len(Basis)):
            for k in range(0,len(Basis)):
                for m in range(0,3):
                    if M == (w**m)*Basis[j].tensor_product(Basis[k]):
                        output_list.append([m,Basis_dic[j], Basis_dic[k]])
    return output_list

In [51]:
def AL_dic(U):
    output_list = []
    for M in AL(U):
        for j in range(0,len(Basis)):
            for k in range(0,len(Basis)):
                for m in range(0,3):
                    if M == (w**m)*Basis[j].tensor_product(Basis[k]):
                        output_list.append([m,Basis_dic[j], Basis_dic[k]])
    return output_list

In [64]:
# U_OLS

for M in AR_dic(U_OLS):
    print(w^M[0], " ", M[1],"x", M[2], "\n")

1   Id3 x Id3 

1   S^2 x S^2 

1   W x W 

1   S x S 

1   W^2 x W^2 

1   S^2 W x S^2 W 

1   S W^2 x S W^2 

1   SW x SW 

1   S^2 W^2 x S^2 W^2 



In [65]:
# U_OLS

for M in AL_dic(U_OLS):
    print(w^M[0], " ", M[1],"x", M[2], "\n")

1   Id3 x Id3 

1   S x S^2 

1   W^2 x W 

1   S^2 x S 

1   W x W^2 

1   S W^2 x S^2 W 

1   S^2 W x S W^2 

1   S^2 W^2 x SW 

1   SW x S^2 W^2 



In [70]:
# U_LambdaCHM

for M in AR_dic(U_LambdaCHM):
    print(w^M[0], " ", M[1],"x", M[2], "\n")

1   Id3 x Id3 

-w - 1   S^2 x SW 

w   W x S^2 W 

-w - 1   S x S^2 W^2 

w   W^2 x S W^2 

-w - 1   S^2 W x W^2 

-w - 1   S W^2 x W 

w   SW x S 

w   S^2 W^2 x S^2 



In [71]:
# U_LambdaCHM 

for M in AL_dic(U_LambdaCHM):
    print(w^M[0], " ", M[1],"x", M[2], "\n")

1   Id3 x Id3 

-w - 1   SW x S^2 

w   S^2 W x W 

-w - 1   S^2 W^2 x S 

w   S W^2 x W^2 

-w - 1   W^2 x S^2 W 

-w - 1   W x S W^2 

w   S x SW 

w   S^2 x S^2 W^2 



In [72]:
# U_Lambda

for M in AR_dic(U_Lambda):
    print(w^M[0], " ", M[1],"x", M[2], "\n")

1   Id3 x Id3 

-w - 1   W^2 x SW 

w   S^2 x S^2 W 

-w - 1   W x S^2 W^2 

w   S x S W^2 

1   S^2 W^2 x W^2 

1   SW x W 

1   S^2 W x S 

1   S W^2 x S^2 



In [73]:
# U_Lambda

for M in AL_dic(U_Lambda):
    print(w^M[0], " ", M[1],"x", M[2], "\n")

1   Id3 x Id3 

-w - 1   SW x W^2 

w   S^2 W x S^2 

-w - 1   S^2 W^2 x W 

w   S W^2 x S 

1   W^2 x S^2 W^2 

1   W x SW 

1   S x S^2 W 

1   S^2 x S W^2 



In [74]:
# U_CHM

for M in AR_dic(U_CHM):
    print(w^M[0], " ", M[1],"x", M[2], "\n")

1   Id3 x Id3 

1   W^2 x S 

1   S^2 x W^2 

1   W x S^2 

1   S x W 

w   S^2 W^2 x S W^2 

w   SW x S^2 W 

-w - 1   S^2 W x S^2 W^2 

-w - 1   S W^2 x SW 



In [75]:
# U_CHM

for M in AL_dic(U_CHM):
    print(w^M[0], " ", M[1],"x", M[2], "\n")

1   Id3 x Id3 

1   S^2 x W 

1   W x S 

1   S x W^2 

1   W^2 x S^2 

w   S^2 W x SW 

w   S W^2 x S^2 W^2 

-w - 1   SW x S W^2 

-w - 1   S^2 W^2 x S^2 W 

